In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re

clubs = {
    "Breeze": "https://unrivaled.basketball/breeze",
    "Hive": "https://unrivaled.basketball/hive",
    "Laces": "https://unrivaled.basketball/laces",
    "Lunar Owls": "https://unrivaled.basketball/lunar-owls",
    "Mist": "https://unrivaled.basketball/mist",
    "Phantom": "https://unrivaled.basketball/phantom",
    "Rose": "https://unrivaled.basketball/rose",
    "Vinyl": "https://unrivaled.basketball/vinyl"
}

headers = {
    "User-Agent": "Mozilla/5.0"
}

# Define a function to normalize names for later merge
def normalize_name(name):
    name = name.upper()
    # Keep only English letters
    name = re.sub(r"[^A-Z]", "", name)
    return name

all_rows = []

for club_name, url in clubs.items():

    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")

    cards = soup.select("a.flex.radius-8")

    for card in cards:
        name_tag = card.select_one("div.text-center")
        if name_tag:
            raw_name = name_tag.get_text(strip=True)
            name_key = normalize_name(raw_name)

            # Remove Satou Sabally only from Phantom (withdrawn)
            if club_name == "Phantom" and name_key == "SATOUSABALLY":
                continue

            all_rows.append([club_name, raw_name, name_key])

# CSV output with NAME_KEY
with open("../data/raw/unrivaled_club_members.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["club", "name", "NAME_KEY"])
    writer.writerows(all_rows)
